In [10]:
# ========== Imputação nested apenas nas variáveis críticas + comparação de submissions ==========
import pandas as pd
import numpy as np
import time, warnings
warnings.filterwarnings("ignore")

BASE = "/Users/augusto/Library/Mobile Documents/com~apple~CloudDocs/git/avalensurance-bia/data"

DF_PATH  = f"{BASE}/full_warehouse_merged.csv"
OUT_PATH = f"{BASE}/full_warehouse_imputed_critical.csv"
SUB1_PATH = f"{BASE}/submission.csv"
SUB2_PATH = f"{BASE}/submission2.csv"

print("DF_PATH :", DF_PATH)
print("OUT_PATH:", OUT_PATH)


# ---------- Carregar df ----------
df = pd.read_csv(DF_PATH)
print("Dataset loaded:", df.shape)

# ---------- Configurações ----------
PROTECTED_COLS = {"annual_medical_cost", "person_id"}
MAX_LEVELS = 3
N_BINS = 10
TOP_K_PREDICTORS = 10

# ---------- Helpers ----------
def is_numeric(s: pd.Series) -> bool:
    return pd.api.types.is_numeric_dtype(s)

def compute_corr_abs(df):
    num = df.select_dtypes(include=[np.number]).copy()
    return num.corr().abs()

def get_top_predictors_for(col, corr_matrix, topk=10):
    if col not in corr_matrix.columns:
        return []
    s = corr_matrix[col].drop(labels=[col], errors="ignore").sort_values(ascending=False)
    return s.index.tolist()[:topk]

def create_binned_cols_local(df_local, cols, n_bins=10):
    binned = {}
    for c in cols:
        if c not in df_local.columns:
            continue
        if is_numeric(df_local[c]):
            try:
                df_local[c + "_bin"] = pd.qcut(df_local[c].rank(method="first"), q=n_bins, labels=False, duplicates="drop")
            except Exception:
                try:
                    df_local[c + "_bin"] = pd.cut(df_local[c], bins=n_bins, labels=False)
                except Exception:
                    df_local[c + "_bin"] = np.nan
            binned[c] = c + "_bin"
        else:
            df_local[c + "_bin"] = df_local[c].astype(str).fillna("missing")
            binned[c] = c + "_bin"
    return binned

def impute_col_nested_medians_local(df_local, col, corr_matrix, max_levels=3, n_bins=10, top_k_predictors=8):
    if col not in df_local.columns or col in PROTECTED_COLS:
        return 0
    n_before = df_local[col].isna().sum()
    if n_before == 0:
        return 0
    top_preds = get_top_predictors_for(col, corr_matrix, topk=top_k_predictors)
    if not top_preds:
        top_preds = [c for c in df_local.select_dtypes(include=[np.number]).columns if c != col][:top_k_predictors]
    top_preds = [p for p in top_preds if p != col and p in df_local.columns]
    binned_map = create_binned_cols_local(df_local, top_preds, n_bins=n_bins)
    for level in range(1, max_levels+1):
        if level > len(top_preds): break
        keys = top_preds[:level]
        key_bins = [binned_map.get(k) for k in keys if binned_map.get(k) is not None]
        if not key_bins: continue
        grouped = df_local.groupby(key_bins)[col].median()
        missing_idx = df_local[df_local[col].isna()].index
        med_values = []
        for idx in missing_idx:
            row = df_local.loc[idx]
            try:
                key = tuple(row[k] for k in key_bins) if len(key_bins) > 1 else row[key_bins[0]]
                m = grouped.loc[key] if len(key_bins) > 1 else grouped.loc[key]
            except Exception:
                m = np.nan
            med_values.append(m)
        med_series = pd.Series(med_values, index=missing_idx)
        to_fill = med_series[med_series.notna()]
        if not to_fill.empty:
            df_local.loc[to_fill.index, col] = to_fill.values
        if df_local[col].isna().sum() == 0:
            break
    # fallback by reference percentile
    if df_local[col].isna().sum() > 0:
        ref = None
        for p in top_preds:
            if is_numeric(df_local[p]):
                ref = p; break
        if ref is None:
            numeric_cols = [c for c in df_local.select_dtypes(include=[np.number]).columns if c != col]
            ref = numeric_cols[0] if numeric_cols else None
        if ref is not None:
            try:
                df_local["__ref_bin__"] = pd.qcut(df_local[ref].rank(method="first"), q=10, labels=False, duplicates="drop")
                grouped_ref = df_local.groupby("__ref_bin__")[col].median()
                missing_idx = df_local[df_local[col].isna()].index
                med_values = df_local.loc[missing_idx, "__ref_bin__"].map(grouped_ref)
                to_fill = med_values[med_values.notna()]
                if not to_fill.empty:
                    df_local.loc[to_fill.index, col] = to_fill.values
            except Exception:
                pass
            df_local.drop(columns=["__ref_bin__"], inplace=True, errors=True)
    # final fallback: median global if numeric
    if is_numeric(df_local[col]) and df_local[col].isna().sum() > 0:
        df_local[col] = df_local[col].fillna(df_local[col].median())
    n_after = df_local[col].isna().sum()
    return n_before - n_after

# ---------- Choose critical columns present in df ----------
candidate = ["bmi","systolic_bp","diastolic_bp","ldl","hba1c","income"]
critical_cols = [c for c in candidate if c in df.columns]
print("Critical columns to impute:", critical_cols)

# ---------- Compute correlation matrix once ----------
corr = compute_corr = df.select_dtypes(include=[np.number]).corr().abs()

# ---------- Run imputations ----------
report = []
t0 = time.time()
for i, col in enumerate(critical_cols, 1):
    t1 = time.time()
    filled = impute_col_nested_medians_local(df, col, corr, max_levels=MAX_LEVELS, n_bins=N_BINS, top_k_predictors=TOP_K_PREDICTORS)
    t2 = time.time()
    report.append((col, filled, df[col].isna().sum(), t2-t1))
    print(f"[{i}/{len(critical_cols)}] {col}: filled={filled}, remaining_nans={df[col].isna().sum()}, time={t2-t1:.1f}s")

print("Total time:", time.time()-t0)

# ---------- Save output ----------
df.to_csv(OUT_PATH, index=False)
print("Saved imputed critical file to:", OUT_PATH)

# ---------- Compare submissions if available ----------
def compare_submissions(sub1_path, sub2_path):
    try:
        s1 = pd.read_csv(sub1_path)
        s2 = pd.read_csv(sub2_path)
    except Exception as e:
        print("Submission files not loaded:", e)
        return None
    # find id column
    id_cols = [c for c in s1.columns if 'id' in c.lower() or 'person' in c.lower()]
    if not id_cols:
        print("No id column found. Returning shapes/cols.")
        return {"s1_shape": s1.shape, "s2_shape": s2.shape, "s1_cols": s1.columns.tolist(), "s2_cols": s2.columns.tolist()}
    idcol = id_cols[0]
    merged = pd.merge(s1, s2, on=idcol, how='inner', suffixes=('_new','_old'))
    # find prediction columns (exclude idcol)
    preds = [c for c in merged.columns if c!=idcol]
    if len(preds) < 2:
        # maybe names equal: take first two non-id columns
        preds = [c for c in merged.columns if c!=idcol][:2]
    pred_new = preds[0]; pred_old = preds[1]
    merged['diff'] = merged[pred_new] - merged[pred_old]
    stats = {
        "n_rows": len(merged),
        "mean_old": merged[pred_old].mean(),
        "mean_new": merged[pred_new].mean(),
        "mean_diff": merged['diff'].mean(),
        "median_diff": merged['diff'].median(),
        "std_diff": merged['diff'].std(),
        "pct_increase": (merged['diff']>0).mean(),
        "top5_increases": merged.sort_values('diff', ascending=False).head(5)[[idcol, pred_old, pred_new, 'diff']].to_dict(orient='records'),
        "top5_decreases": merged.sort_values('diff').head(5)[[idcol, pred_old, pred_new, 'diff']].to_dict(orient='records')
    }
    return stats

comp = compare_submissions(SUB1_PATH, SUB2_PATH)
print("Comparison:", comp)

# ---------- Return report DataFrame ----------
report_df = pd.DataFrame(report, columns=['col','filled','remaining_nans','time_s'])
report_df


DF_PATH : /Users/augusto/Library/Mobile Documents/com~apple~CloudDocs/git/avalensurance-bia/data/full_warehouse_merged.csv
OUT_PATH: /Users/augusto/Library/Mobile Documents/com~apple~CloudDocs/git/avalensurance-bia/data/full_warehouse_imputed_critical.csv
Dataset loaded: (64800, 59)
Critical columns to impute: ['bmi', 'systolic_bp', 'diastolic_bp', 'ldl', 'hba1c', 'income']
[1/6] bmi: filled=0, remaining_nans=0, time=0.0s
[2/6] systolic_bp: filled=0, remaining_nans=0, time=0.0s
[3/6] diastolic_bp: filled=0, remaining_nans=0, time=0.0s
[4/6] ldl: filled=0, remaining_nans=0, time=0.0s
[5/6] hba1c: filled=0, remaining_nans=0, time=0.0s
[6/6] income: filled=0, remaining_nans=0, time=0.0s
Total time: 0.0074310302734375
Saved imputed critical file to: /Users/augusto/Library/Mobile Documents/com~apple~CloudDocs/git/avalensurance-bia/data/full_warehouse_imputed_critical.csv
Comparison: {'n_rows': 7200, 'mean_old': np.float64(2880.7365777864647), 'mean_new': np.float64(1929.2105288611112), 'mea

,col,filled,remaining_nans,time_s
0,bmi,0,0,0.001854
1,systolic_bp,0,0,0.002844
2,diastolic_bp,0,0,0.000091
3,ldl,0,0,0.000082
4,hba1c,0,0,0.000079
5,income,0,0,0.000083
